In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_12 = pd.read_csv('data/actuals_predictions_state_2012.csv', index_col=0)
df_16 = pd.read_csv('data/actuals_predictions_state_2016.csv', index_col=0)

In [3]:
df_12['TOP_TOPIC'].unique()

array(['Economy', 'Immigration', 'Healthcare'], dtype=object)

In [4]:
df_12.dtypes

STATE_FIPS              int64
PRED_DEM_VOTES          int64
PRED_REP_VOTES          int64
PRED_WINNING_PARTY      int64
STATE                  object
TOTAL_DEM_VOTES        object
TOTAL_REP_VOTES        object
TOTAL_VOTES            object
MARGIN_VICTORY        float64
ABS_MARGIN_VICTORY    float64
WINNING_PARTY           int64
ELECTORAL_VOTES         int64
TOP_TOPIC              object
dtype: object

In [5]:
df_12[["PRED_DEM_VOTES", "PRED_REP_VOTES"]] = df_12[["PRED_DEM_VOTES", "PRED_REP_VOTES"]].apply(pd.to_numeric)
df_16[["PRED_DEM_VOTES", "PRED_REP_VOTES"]] = df_16[["PRED_DEM_VOTES", "PRED_REP_VOTES"]].apply(pd.to_numeric)

df_12['PRED_TOTAL_VOTES'] = df_12['PRED_DEM_VOTES'] + df_12['PRED_REP_VOTES']
df_16['PRED_TOTAL_VOTES'] = df_16['PRED_DEM_VOTES'] + df_16['PRED_REP_VOTES']

In [6]:
def get_winner_by_topic_and_year(tp, yr):
    if tp == 'Economy':
        if yr == '2012':
            return 'r'
        if yr == '2016':
            return 'r'
    elif tp == 'Immigration':
        if yr == '2012':
            return 'd'
        if yr == '2016':
            return 'd'
    elif tp == 'Healthcare':
        if yr == '2012':
            return 'r'
        if yr == '2016':
            return 'd'

In [18]:
#TODO - Get the new winning margin

def adjust_weights_and_get_acc(df, weight, yr):

    df2 = df[['STATE_FIPS', 'STATE', 'WINNING_PARTY', 'PRED_WINNING_PARTY', 'PRED_DEM_VOTES', 'PRED_REP_VOTES']]
    
    acc_before = accuracy_score(df2['WINNING_PARTY'], df2['PRED_WINNING_PARTY'])

    for idx, row in df.iterrows():
        dem_votes = 0
        rep_votes = 0

        if get_winner_by_topic_and_year(row['TOP_TOPIC'], yr) == 'd':
            dem_votes = row['PRED_DEM_VOTES'] + (row['PRED_TOTAL_VOTES'] * weight)
            rep_votes = row['PRED_REP_VOTES'] - (row['PRED_TOTAL_VOTES'] * weight)
        else:
            dem_votes = row['PRED_DEM_VOTES'] - (row['PRED_TOTAL_VOTES'] * weight)
            rep_votes = row['PRED_REP_VOTES'] + (row['PRED_TOTAL_VOTES'] * weight)

        df2.loc[idx,'ADJUSTED_DEM_VOTES'] = dem_votes
        df2.loc[idx,'ADJUSTED_REP_VOTES'] = rep_votes
            
        if dem_votes > rep_votes and row['PRED_WINNING_PARTY'] == 1:
            print('rep->dem', row['STATE'])
            df2.loc[idx,'ADJUSTED_WINNING_PARTY'] = 0
            df2.loc[idx,'WINNER_SHIFT'] = 'REP->DEM'
        elif dem_votes < rep_votes and row['PRED_WINNING_PARTY'] == 0:
            print('dem->rep', row['STATE'])
            df2.loc[idx,'ADJUSTED_WINNING_PARTY'] = 1
            df2.loc[idx,'WINNER_SHIFT'] = 'DEM->REP'
        else:
            df2.loc[idx,'ADJUSTED_WINNING_PARTY'] = row['PRED_WINNING_PARTY']
 
    acc_after = accuracy_score(df2['WINNING_PARTY'], df2['ADJUSTED_WINNING_PARTY'])
    
    return acc_before, acc_after, df2

In [24]:
start = 0.025
acc_before, acc_after, data = adjust_weights_and_get_acc(df_12, start, '2012')
print(acc_before, acc_after)
data.head(50)

data.to_csv('data/predictions_vs_adjusted_2012.csv')

#for factor in range(100):
    
#    ab, af = adjust_weights_and_get_acc(df_12, start)
#    start = start + 0.0013
    
   # print(af - ab)

#adjust_weights_and_get_acc(df_12)


dem->rep Alaska
dem->rep Ohio
dem->rep Oregon
dem->rep Texas
0.8823529411764706 0.8823529411764706


In [25]:

acc_before, acc_after, data = adjust_weights_and_get_acc(df_16, start, '2016')
print(acc_before, acc_after)
data.to_csv('data/predictions_vs_adjusted_2016.csv')
data.head(50)

dem->rep Georgia
dem->rep Ohio
dem->rep Texas
rep->dem Virginia
0.8235294117647058 0.9019607843137255


,STATE_FIPS,STATE,WINNING_PARTY,PRED_WINNING_PARTY,PRED_DEM_VOTES,PRED_REP_VOTES,ADJUSTED_DEM_VOTES,ADJUSTED_REP_VOTES,ADJUSTED_WINNING_PARTY,WINNER_SHIFT
0,1,Alabama,1,1,969055,1127406,916643.475,1179817.525,1.0,NaN
1,2,Alaska,1,1,216755,239290,205353.875,250691.125,1.0,NaN
2,4,Arizona,1,0,1616743,789572,1556585.125,849729.875,0.0,NaN
3,5,Arkansas,1,1,454001,710259,424894.500,739365.500,1.0,NaN
4,6,California,0,0,10058342,4061414,9705348.100,4414407.900,0.0,NaN
5,8,Colorado,0,0,1282494,988758,1339275.300,931976.700,0.0,NaN
6,9,Connecticut,0,0,1092149,614594,1049480.425,657262.575,0.0,NaN
7,10,Delaware,0,0,266303,160484,255633.325,171153.675,0.0,NaN
8,11,District of Columbia,0,0,169442,98537,162742.525,105236.475,0.0,NaN
9,12,Florida,1,0,4474914,3702272,4679343.650,3497842.350,0.0,NaN
